# Machine Learning Course: Feature Selection

## Notebook 3: Advanced Feature Selection for High-Dimensional Genomics Data

> **📋 Prerequisites:** Complete `01_data_exploration.ipynb` and `02_data_preprocessing.ipynb` first. This notebook uses the preprocessed data from notebook 2.

### Learning Objectives
By the end of this notebook, you will be able to:
1. **Apply advanced variance filtering** beyond basic preprocessing thresholds
2. **Remove highly correlated features** to reduce redundancy
3. **Use differential expression analysis** to identify category-specific genes
4. **Implement Random Forest importance** with KneeLocator for optimal feature selection
5. **Apply alternative ML-based importance** methods (LASSO, SVM, etc.)
6. **Use Boruta algorithm** for all-relevant feature detection
7. **Apply Cox regression** for survival-based feature association

### Data Flow
This notebook refines features from preprocessed data:
- **Input**: Preprocessed, scaled datasets from `02_data_preprocessing.ipynb`
- **Processing**: Seven advanced feature selection methods
- **Output**: Optimized feature sets for final model development

### Feature Selection Philosophy
Unlike preprocessing (notebook 2) which focused on data quality and basic filtering, this notebook applies **advanced statistical and machine learning methods** to identify the most **predictive and biologically relevant** features for classification and survival prediction.

---

## 1. Load Preprocessed Data

Load the cleaned and preprocessed data from notebook 2.

In [ ]:
# 📝 ACTIVITY 1: Load Preprocessed Data and Import Libraries
#
# Your task: Load the preprocessed results from notebook 2 and import feature selection libraries
#
# TODO: Import required libraries:
# 1. Import pandas, numpy, matplotlib.pyplot, seaborn
# 2. Import sklearn feature_selection: VarianceThreshold, SelectKBest, f_classif, mutual_info_classif
# 3. Import sklearn models: RandomForestClassifier, LogisticRegression (with Lasso penalty)
# 4. Import sklearn.svm: LinearSVC
# 5. Import scipy.stats: ttest_ind, mannwhitneyu
# 6. Try importing: from kneed import KneeLocator (for knee detection)
# 7. Try importing: from boruta import BorutaPy (optional)
# 8. Try importing: from lifelines import CoxPHFitter (for survival analysis)
#
# TODO: Load preprocessed data:
# 9. Load from '../results/preprocessing/':
#    - X_train_scaled.csv, X_val_scaled.csv, X_test_scaled.csv
#    - y_train.csv, y_val.csv, y_test.csv
# 10. Load clinical data if available:
#     - clinical_train.csv, clinical_val.csv, clinical_test.csv
# 11. Print shapes and confirm data is loaded correctly
#
# TODO: Verify data from preprocessing:
# 12. Check that target variables include RFS_MONTHS and RFS_STATUS (from notebook 2)
# 13. Confirm that gene expression data is normalized and filtered (from notebook 2)
# 14. Display basic statistics to validate data quality
#
# Expected output: Loaded preprocessed data and imported feature selection libraries

# Write your code below:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# 
# # Load preprocessed data
# preprocessing_dir = '../results/preprocessing/'
# X_train = pd.read_csv(f'{preprocessing_dir}X_train_scaled.csv', index_col=0)
# X_val = pd.read_csv(f'{preprocessing_dir}X_val_scaled.csv', index_col=0)
# X_test = pd.read_csv(f'{preprocessing_dir}X_test_scaled.csv', index_col=0)
# y_train = pd.read_csv(f'{preprocessing_dir}y_train.csv', index_col=0)
# ...

## 2. Advanced Variance-Based Feature Selection

Apply stricter variance filtering than in preprocessing to identify genes with the highest variation.

In [ ]:
# 📝 ACTIVITY 2: Advanced Variance Filtering (Stronger than Notebook 2)
#
# Your task: Apply more aggressive variance filtering to select highly variable genes
#
# TODO: Calculate variance statistics:
# 1. Calculate variance for all features: gene_variances = X_train.var()
# 2. Calculate coefficient of variation: gene_cv = X_train.std() / X_train.mean().abs()
# 3. Calculate percentiles: np.percentile(gene_variances, [25, 50, 75, 90, 95])
# 4. Print variance distribution summary
#
# TODO: Apply stronger variance threshold:
# 5. In notebook 2, you may have used threshold ~0.01-0.05
# 6. Now use higher threshold (e.g., 0.1 or top 75th percentile) to select only highly variable genes
# 7. Create VarianceThreshold selector: selector = VarianceThreshold(threshold=0.1)
# 8. Fit and transform: selector.fit(X_train)
# 9. Get selected features: variance_selected_features = X_train.columns[selector.get_support()]
#
# TODO: Alternative: Select top N most variable genes:
# 10. Sort genes by variance: sorted_by_variance = gene_variances.sort_values(ascending=False)
# 11. Select top 1000 or top 50%: top_variance_genes = sorted_by_variance.head(1000).index
# 12. Compare fixed threshold vs top-N selection
#
# TODO: Visualize variance distribution:
# 13. Create histogram of gene variances
# 14. Mark threshold on the plot
# 15. Show number of genes selected vs removed
#
# TODO: Print summary:
# 16. Original number of genes (after preprocessing)
# 17. Number of genes after advanced variance filtering
# 18. Percentage of genes retained
# 19. Show variance statistics for selected vs removed genes
#
# Expected output: Highly variable genes selected with stricter threshold than preprocessing

# Write your code below:
# print("ADVANCED VARIANCE FILTERING")
# print("="*60)
# gene_variances = X_train.var()
# ...

## 3. De-correlation: Remove Redundant Features

Remove highly correlated features to reduce redundancy and multicollinearity.

In [ ]:
# 📝 ACTIVITY 3: De-correlation - Remove Highly Correlated Features
#
# Your task: Identify and remove redundant features with high pairwise correlations
#
# TODO: Calculate feature-feature correlation matrix:
# 1. Start with variance-filtered features from Activity 2
# 2. Calculate correlation matrix: corr_matrix = X_train[variance_selected_features].corr()
# 3. Get absolute correlations: abs_corr_matrix = corr_matrix.abs()
#
# TODO: Identify highly correlated pairs:
# 4. Set correlation threshold (e.g., 0.8 or 0.9 for high correlation)
# 5. Find upper triangle of correlation matrix to avoid duplicates:
#    - upper_triangle = np.triu(abs_corr_matrix, k=1)
# 6. Find pairs with correlation above threshold
# 7. Create list of correlated feature pairs
#
# TODO: Remove redundant features:
# 8. For each highly correlated pair, decide which to keep:
#    - Option 1: Keep the one with higher variance
#    - Option 2: Keep the one with higher correlation to target
#    - Option 3: Keep the first one alphabetically (simplest)
# 9. Create list of features to drop: features_to_drop = []
# 10. Remove redundant features: decorrelated_features = [f for f in variance_selected_features if f not in features_to_drop]
#
# TODO: Alternative: Use clustering approach:
# 11. Cluster features by correlation
# 12. Select one representative from each cluster
#
# TODO: Visualize correlations:
# 13. Create heatmap of correlation matrix (sample of features)
# 14. Show distribution of pairwise correlations
# 15. Highlight features that were removed
#
# TODO: Print de-correlation summary:
# 16. Number of highly correlated pairs found
# 17. Number of features removed due to redundancy
# 18. Number of features remaining after de-correlation
# 19. Show examples of correlated pairs that were addressed
#
# Expected output: De-correlated feature set with redundancy removed

# Write your code below:
# print("DE-CORRELATION: REMOVING REDUNDANT FEATURES")
# print("="*60)
# corr_matrix = X_train[variance_selected_features].corr()
# ...

## 4. Differential Gene Expression Between Categories

Identify genes that are significantly different between outcome categories using statistical tests.

In [ ]:
# 📝 ACTIVITY 4: Differential Gene Expression Analysis
#
# Your task: Identify genes with significant expression differences between outcome categories
#
# TODO: Prepare data for differential expression:
# 1. Start with de-correlated features from Activity 3
# 2. Define outcome categories from y_train (e.g., RFS_STATUS: 0=no recurrence, 1=recurrence)
# 3. Split samples by category: group_0 = X_train[y_train['RFS_STATUS'] == 0]
#                               group_1 = X_train[y_train['RFS_STATUS'] == 1]
#
# TODO: Calculate differential expression statistics:
# 4. For each gene, calculate:
#    - Mean expression in group 0 and group 1
#    - Log2 fold change: log2(mean_group1 / mean_group0)
#    - Handle zero/negative values appropriately
# 5. Calculate statistical significance using t-test:
#    - For each gene: t_stat, p_value = ttest_ind(group_0[gene], group_1[gene])
# 6. Store results in DataFrame: de_results with columns [gene, log2fc, p_value, mean_group0, mean_group1]
#
# TODO: Apply multiple testing correction:
# 7. Import: from statsmodels.stats.multitest import multipletests
# 8. Correct p-values: reject, pvals_corrected, _, _ = multipletests(de_results['p_value'], method='fdr_bh')
# 9. Add corrected p-values to results: de_results['p_adj'] = pvals_corrected
#
# TODO: Select differentially expressed genes:
# 10. Set thresholds:
#     - p_adj < 0.05 (statistical significance after correction)
#     - abs(log2fc) > 1 (at least 2-fold change)
# 11. Select significant genes: de_genes = de_results[(de_results['p_adj'] < 0.05) & (de_results['log2fc'].abs() > 1)]
# 12. Get list of DE gene names: de_selected_features = de_genes.index.tolist()
#
# TODO: Alternative: Use Mann-Whitney U test (non-parametric):
# 13. For non-normally distributed data, use mannwhitneyu instead of ttest_ind
# 14. Compare results between parametric and non-parametric tests
#
# TODO: Visualize differential expression:
# 15. Create volcano plot: x=log2fc, y=-log10(p_adj), color by significance
# 16. Create MA plot: x=mean expression, y=log2fc
# 17. Show distribution of fold changes and p-values
# 18. Highlight top differentially expressed genes
#
# TODO: Analyze DE results:
# 19. Count upregulated genes (log2fc > 1) and downregulated genes (log2fc < -1)
# 20. Show top 10 most significant genes with largest fold changes
# 21. Compare mean expression levels between groups for top genes
#
# TODO: Print DE summary:
# 22. Total genes tested
# 23. Number of significant genes at different thresholds
# 24. Distribution of fold changes
# 25. Examples of top differentially expressed genes
#
# Expected output: Differentially expressed genes between outcome categories

# Write your code below:
# print("DIFFERENTIAL GENE EXPRESSION ANALYSIS")
# print("="*60)
# from scipy.stats import ttest_ind
# ...

## 5. Random Forest Importance with KneeLocator

Use Random Forest to rank features by importance and apply KneeLocator to find optimal cutoff.

In [ ]:
# 📝 ACTIVITY 5: Random Forest Importance with KneeLocator
#
# Your task: Use Random Forest to rank features and find optimal number using the elbow method
#
# TODO: Train Random Forest on all current features:
# 1. Use features from previous activities (or all preprocessed features)
# 2. Create RandomForestClassifier: rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
# 3. Fit on training data: rf.fit(X_train[current_features], y_train['RFS_STATUS'])
# 4. Extract feature importances: rf_importances = pd.Series(rf.feature_importances_, index=current_features)
#
# TODO: Sort features by importance:
# 5. Sort importances descending: rf_ranking = rf_importances.sort_values(ascending=False)
# 6. Calculate cumulative importance: cumulative_importance = rf_ranking.cumsum()
# 7. Normalize cumulative importance: cumulative_importance / cumulative_importance.max()
#
# TODO: Apply KneeLocator to find optimal cutoff:
# 8. Check if kneed library is available:
#    try:
#        from kneed import KneeLocator
# 9. Create x values (feature ranks) and y values (sorted importances)
# 10. Apply KneeLocator: kl = KneeLocator(x=range(len(rf_ranking)), y=rf_ranking.values, curve='convex', direction='decreasing')
# 11. Find knee point: knee_point = kl.knee
# 12. Select features up to knee point: rf_knee_features = rf_ranking.iloc[:knee_point].index.tolist()
#
# TODO: Alternative cutoff methods if KneeLocator unavailable:
# 13. Select features contributing to 90% cumulative importance
# 14. Select top N features (e.g., top 100 or 200)
# 15. Select features with importance > threshold (e.g., > 0.001)
#
# TODO: Validate Random Forest selection:
# 16. Train new RF with only selected features
# 17. Evaluate on validation set: accuracy, AUC
# 18. Compare performance with all features vs selected features
#
# TODO: Visualize Random Forest importance:
# 19. Plot feature importance curve with knee point marked
# 20. Plot cumulative importance curve
# 21. Show bar plot of top 20 most important features
# 22. Create importance distribution histogram
#
# TODO: Print RF importance summary:
# 23. Total features evaluated
# 24. Knee point location (feature rank)
# 25. Number of features selected by knee method
# 26. Cumulative importance of selected features
# 27. Show top 20 features with their importance scores
#
# Expected output: Optimal feature set identified using Random Forest importance and KneeLocator

# Write your code below:
# print("RANDOM FOREST IMPORTANCE WITH KNEELOCATOR")
# print("="*60)
# from sklearn.ensemble import RandomForestClassifier
# ...

## 6. Other ML-Based Feature Importance

Apply LASSO (L1 regularization) and SVM-based feature selection methods.

In [ ]:
# 📝 ACTIVITY 6: LASSO and SVM-Based Feature Selection
#
# Your task: Use L1-regularized models to identify important features through embedded selection
#
# TODO: Apply LASSO (L1-regularized Logistic Regression):
# 1. Import: from sklearn.linear_model import LogisticRegression, LassoCV
# 2. Create L1-penalized logistic regression: lasso_lr = LogisticRegression(penalty='l1', solver='liblinear', C=0.1, random_state=42)
# 3. Fit on training data: lasso_lr.fit(X_train[current_features], y_train['RFS_STATUS'])
# 4. Extract coefficients: lasso_coefs = pd.Series(lasso_lr.coef_[0], index=current_features)
# 5. Get non-zero coefficients: lasso_selected = lasso_coefs[lasso_coefs != 0].index.tolist()
#
# TODO: Optimize LASSO regularization strength:
# 6. Try different C values: C_values = [0.001, 0.01, 0.1, 1, 10]
# 7. For each C, count number of selected features
# 8. Plot: C vs number of selected features
# 9. Choose optimal C that balances sparsity and performance
#
# TODO: Apply Linear SVM with L1 penalty:
# 10. Import: from sklearn.svm import LinearSVC
# 11. Create LinearSVC with L1 penalty: svm_l1 = LinearSVC(penalty='l1', dual=False, C=0.1, random_state=42, max_iter=10000)
# 12. Fit on training data: svm_l1.fit(X_train[current_features], y_train['RFS_STATUS'])
# 13. Extract coefficients: svm_coefs = pd.Series(svm_l1.coef_[0], index=current_features)
# 14. Get non-zero coefficients: svm_selected = svm_coefs[svm_coefs != 0].index.tolist()
#
# TODO: Use SelectFromModel wrapper:
# 15. Import: from sklearn.feature_selection import SelectFromModel
# 16. Create selector: selector = SelectFromModel(LogisticRegression(penalty='l1', solver='liblinear', C=0.1))
# 17. Fit and transform: selector.fit(X_train[current_features], y_train['RFS_STATUS'])
# 18. Get selected features: sfm_selected = current_features[selector.get_support()]
#
# TODO: Compare LASSO and SVM selections:
# 19. Calculate overlap between LASSO and SVM selected features
# 20. Identify features selected by both methods (intersection)
# 21. Identify features selected by either method (union)
# 22. Create combined ML-based feature set: ml_selected_features
#
# TODO: Analyze coefficient patterns:
# 23. Compare coefficient magnitudes between LASSO and SVM
# 24. Identify features with consistently large coefficients
# 25. Plot coefficient distributions
#
# TODO: Visualize ML-based selection:
# 26. Create bar plots of top coefficients for LASSO and SVM
# 27. Create scatter plot: LASSO coef vs SVM coef
# 28. Show regularization path (coefficient vs C values)
#
# TODO: Validate ML selections:
# 29. Train models with selected features on train set
# 30. Evaluate on validation set: accuracy, AUC
# 31. Compare LASSO vs SVM vs combined selection performance
#
# TODO: Print ML selection summary:
# 32. Number of features selected by LASSO
# 33. Number of features selected by SVM
# 34. Number of features in intersection and union
# 35. Show top 20 features by absolute coefficient magnitude
#
# Expected output: Feature selection via LASSO and SVM with coefficient analysis

# Write your code below:
# print("ML-BASED FEATURE SELECTION: LASSO AND SVM")
# print("="*60)
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC
# ...

## 7. Boruta All-Relevant Feature Selection

Apply Boruta algorithm to identify all features relevant to the prediction task.

In [ ]:
# 📝 ACTIVITY 7: Boruta All-Relevant Feature Selection
#
# Your task: Use Boruta algorithm to identify all statistically relevant features
#
# TODO: Check Boruta availability:
# 1. Try importing: from boruta import BorutaPy
# 2. If not available, print installation instructions: pip install Boruta
# 3. Handle case where Boruta is not installed gracefully
#
# TODO: Prepare data for Boruta:
# 4. Convert to numpy arrays: X_array = X_train[current_features].values
# 5. Convert target: y_array = y_train['RFS_STATUS'].values
# 6. Ensure data types are appropriate
#
# TODO: Configure and run Boruta:
# 7. Create base estimator: rf_boruta = RandomForestClassifier(n_jobs=-1, max_depth=5, random_state=42)
# 8. Create Boruta selector:
#    boruta = BorutaPy(
#        estimator=rf_boruta,
#        n_estimators=100,
#        max_iter=100,
#        random_state=42,
#        verbose=2
#    )
# 9. Fit Boruta: boruta.fit(X_array, y_array)
#
# TODO: Extract Boruta results:
# 10. Get confirmed features: boruta_confirmed = current_features[boruta.support_].tolist()
# 11. Get tentative features: boruta_tentative = current_features[boruta.support_weak_].tolist()
# 12. Get rejected features: boruta_rejected = current_features[~(boruta.support_ | boruta.support_weak_)].tolist()
# 13. Get feature rankings: boruta_ranking = pd.Series(boruta.ranking_, index=current_features)
#
# TODO: Decide on tentative features:
# 14. Option 1: Include tentative features (liberal approach)
# 15. Option 2: Exclude tentative features (conservative approach)
# 16. Create final Boruta feature set: boruta_selected_features = boruta_confirmed (or + tentative)
#
# TODO: Analyze Boruta iterations:
# 17. If available, analyze feature importance history across iterations
# 18. Identify features that were quickly confirmed vs those that took many iterations
# 19. Check convergence behavior
#
# TODO: Visualize Boruta results:
# 20. Create bar plot showing confirmed, tentative, and rejected counts
# 21. Plot feature importance for confirmed features
# 22. Show Boruta ranking distribution
# 23. If history available, plot importance evolution over iterations
#
# TODO: Compare with shadow features:
# 24. If shadow feature importances are accessible, compare with real features
# 25. Show why certain features were confirmed or rejected
#
# TODO: Print Boruta summary:
# 26. Number of confirmed features
# 27. Number of tentative features
# 28. Number of rejected features
# 29. Show list of top confirmed features
# 30. Explain decision on tentative features
#
# Expected output: All-relevant features identified via Boruta algorithm

# Write your code below:
# print("BORUTA ALL-RELEVANT FEATURE SELECTION")
# print("="*60)
#
# try:
#     from boruta import BorutaPy
#     boruta_available = True
#     print("✓ Boruta library available")
# except ImportError:
#     boruta_available = False
#     print("⚠️  Boruta not installed")
#     print("To install: pip install Boruta")
# ...

In [ ]:
# 📝 ACTIVITY 9: Method Comparison and Final Feature Set Selection
#
# Your task: Compare all 7 methods and create optimized ensemble feature sets
#
# TODO: Collect all method results:
# 1. Gather feature sets from all activities:
#    - variance_selected_features (Activity 2)
#    - decorrelated_features (Activity 3)
#    - de_selected_features (Activity 4)
#    - rf_knee_features (Activity 5)
#    - ml_selected_features (Activity 6)
#    - boruta_selected_features (Activity 7)
#    - cox_selected_features (Activity 8)
# 2. Handle any missing results if a method wasn't completed
#
# TODO: Create method comparison matrix:
# 3. Get all unique features across methods: all_features = set().union(*all_feature_sets)
# 4. Create binary matrix: rows=features, columns=methods, values=1 if selected, 0 otherwise
# 5. Calculate vote count for each feature: vote_counts = matrix.sum(axis=1)
#
# TODO: Calculate method statistics:
# 6. For each method, count number of features selected
# 7. Calculate pairwise overlap (Jaccard similarity) between methods
# 8. Create method similarity matrix
# 9. Identify which methods agree most/least
#
# TODO: Create ensemble feature sets with different strategies:
# 10. **Unanimous**: Features selected by all 7 methods (intersection)
# 11. **Strong Consensus**: Features selected by ≥5 methods (71%+)
# 12. **Majority Consensus**: Features selected by ≥4 methods (57%+)
# 13. **Liberal Consensus**: Features selected by ≥3 methods (43%+)
# 14. **Union**: Features selected by any method (union of all)
#
# TODO: Validate ensemble feature sets:
# 15. For each ensemble strategy:
#     - Train RandomForestClassifier on train set
#     - Evaluate on validation set: accuracy, precision, recall, F1, AUC
#     - Compare feature set size vs performance
# 16. Identify optimal trade-off: performance vs number of features
#
# TODO: Create stability analysis:
# 17. Calculate feature selection frequency across all methods
# 18. Identify most stable features (selected by many methods)
# 19. Identify method-specific features (selected by only one method)
#
# TODO: Visualize method comparison:
# 20. Create UpSet plot or Venn diagram showing method overlaps
# 21. Create heatmap of method similarity matrix
# 22. Plot feature set size vs validation performance
# 23. Show vote count distribution across features
# 24. Create bar plot: number of features per method
#
# TODO: Recommend final feature sets:
# 25. **Minimal Set**: Unanimous features (highest confidence, smallest size)
# 26. **Balanced Set**: Majority consensus (good balance of size vs coverage)
# 27. **Comprehensive Set**: Liberal consensus (more features, more coverage)
# 28. Provide rationale for each recommendation based on use case
#
# TODO: Print comparison summary:
# 29. Table showing: Method | # Features | Overlap with Others
# 30. Show feature counts for each ensemble strategy
# 31. Display validation performance for each ensemble
# 32. List top 20 features by vote count
# 33. Recommend which ensemble to use for final modeling
#
# TODO: Export results:
# 34. Save all method results to '../results/feature_selection/'
# 35. Export ensemble feature sets (minimal, balanced, comprehensive)
# 36. Save method comparison matrices
# 37. Create comprehensive feature selection report (JSON or CSV)
# 38. Save visualizations
#
# Expected output: Complete method comparison with optimal ensemble feature sets ready for modeling

# Write your code below:
# print("FEATURE SELECTION METHOD COMPARISON")
# print("="*60)
#
# # Collect all feature sets
# all_methods = {
#     'Variance': variance_selected_features,
#     'Decorrelation': decorrelated_features,
#     'Diff_Expression': de_selected_features,
#     'RandomForest': rf_knee_features,
#     'ML_Methods': ml_selected_features,
#     'Boruta': boruta_selected_features,
#     'Cox': cox_selected_features
# }
# ...

## 9. Compare Methods and Create Final Feature Set

Compare all 7 feature selection methods and create optimal ensemble feature sets.

In [ ]:
# 📝 ACTIVITY 8: Cox Regression for Survival-Based Feature Association
#
# Your task: Identify genes associated with survival outcomes using Cox proportional hazards models
#
# TODO: Check lifelines availability:
# 1. Try importing: from lifelines import CoxPHFitter
# 2. If not available, provide alternative statistical approach
# 3. Print installation instructions if needed: pip install lifelines
#
# TODO: Prepare survival data:
# 4. Extract survival time: duration = y_train['RFS_MONTHS']
# 5. Extract event indicator: event = y_train['RFS_STATUS']
# 6. Handle missing values in survival data
# 7. Ensure duration > 0 (Cox requirement)
#
# TODO: Perform univariate Cox regression for each gene:
# 8. Initialize results storage: cox_results = {}
# 9. For each gene in current_features:
#    - Create DataFrame: df = pd.DataFrame({'duration': duration, 'event': event, 'gene': X_train[gene]})
#    - Fit Cox model: cph = CoxPHFitter(); cph.fit(df, duration_col='duration', event_col='event')
#    - Extract hazard ratio: hr = cph.hazard_ratios_['gene']
#    - Extract p-value: p_val = cph.summary.loc['gene', 'p']
#    - Store results: cox_results[gene] = {'hr': hr, 'p_value': p_val}
# 10. Handle fitting errors gracefully (some genes may fail to converge)
#
# TODO: Compile Cox results:
# 11. Create DataFrame: cox_df = pd.DataFrame(cox_results).T
# 12. Calculate log hazard ratios: cox_df['log_hr'] = np.log(cox_df['hr'])
# 13. Calculate -log10(p-value): cox_df['neg_log_p'] = -np.log10(cox_df['p_value'])
# 14. Sort by statistical significance
#
# TODO: Apply multiple testing correction:
# 15. Import: from statsmodels.stats.multitest import multipletests
# 16. Correct p-values: reject, p_adj, _, _ = multipletests(cox_df['p_value'], method='fdr_bh')
# 17. Add to results: cox_df['p_adj'] = p_adj
#
# TODO: Select significant genes:
# 18. Set thresholds:
#     - p_adj < 0.05 (statistical significance)
#     - HR > 1.5 or HR < 0.67 (meaningful effect size: 50% increase/decrease in hazard)
# 19. Select significant genes: cox_significant = cox_df[(cox_df['p_adj'] < 0.05) & ((cox_df['hr'] > 1.5) | (cox_df['hr'] < 0.67))]
# 20. Get selected features: cox_selected_features = cox_significant.index.tolist()
#
# TODO: Alternative if lifelines unavailable:
# 21. Use correlation between gene expression and survival time
# 22. Use logistic regression with survival status as binary outcome
# 23. Apply statistical tests comparing expression in patients with events vs without
#
# TODO: Visualize Cox results:
# 24. Create volcano plot: x=log(HR), y=-log10(p_adj), color by significance
# 25. Create forest plot showing hazard ratios with confidence intervals (if available)
# 26. Show distribution of hazard ratios
# 27. Highlight top genes with strongest associations
#
# TODO: Interpret Cox results:
# 28. Identify protective genes (HR < 1): lower expression = higher risk
# 29. Identify risk genes (HR > 1): higher expression = higher risk
# 30. Compare biological interpretation with clinical expectations
#
# TODO: Print Cox summary:
# 31. Total genes tested
# 32. Number of significant genes after correction
# 33. Number of protective vs risk genes
# 34. Show top 20 genes by hazard ratio and significance
# 35. Report genes with clinical relevance if known
#
# Expected output: Survival-associated genes identified via Cox regression

# Write your code below:
# print("COX REGRESSION-BASED SURVIVAL ASSOCIATION")
# print("="*60)
#
# try:
#     from lifelines import CoxPHFitter
#     lifelines_available = True
#     print("✓ Lifelines library available")
# except ImportError:
#     lifelines_available = False
#     print("⚠️  Lifelines not installed")
#     print("To install: pip install lifelines")
#     print("Using alternative statistical approach...")
# ...

## 8. Cox Regression-Based Association

Use survival analysis to identify genes associated with recurrence-free survival.

## 💡 Coding Hints and Templates

Need help getting started? Here are code templates for each feature selection method:

### 📋 **Template 1: Advanced Variance Filtering**
```python
from sklearn.feature_selection import VarianceThreshold
gene_variances = X_train.var()
threshold = np.percentile(gene_variances, 75)  # Top 25%
selector = VarianceThreshold(threshold=threshold)
selector.fit(X_train)
variance_selected = X_train.columns[selector.get_support()]
```

### 📋 **Template 2: De-correlation**
```python
corr_matrix = X_train.corr().abs()
upper = np.triu(corr_matrix, k=1)
to_drop = [column for column in corr_matrix.columns 
           if any(upper[corr_matrix.columns.get_loc(column)] > 0.85)]
decorrelated_features = [f for f in X_train.columns if f not in to_drop]
```

### 📋 **Template 3: Differential Expression**
```python
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

de_results = []
for gene in X_train.columns:
    group0 = X_train.loc[y_train['RFS_STATUS'] == 0, gene]
    group1 = X_train.loc[y_train['RFS_STATUS'] == 1, gene]
    t_stat, p_val = ttest_ind(group0, group1)
    log2fc = np.log2((group1.mean() + 1e-10) / (group0.mean() + 1e-10))
    de_results.append({'gene': gene, 'log2fc': log2fc, 'p_value': p_val})

de_df = pd.DataFrame(de_results).set_index('gene')
_, p_adj, _, _ = multipletests(de_df['p_value'], method='fdr_bh')
de_df['p_adj'] = p_adj
de_selected = de_df[(de_df['p_adj'] < 0.05) & (de_df['log2fc'].abs() > 1)].index
```

### 📋 **Template 4: Random Forest with KneeLocator**
```python
from sklearn.ensemble import RandomForestClassifier
from kneed import KneeLocator

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train['RFS_STATUS'])
importances = pd.Series(rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

kl = KneeLocator(range(len(importances)), importances.values, 
                 curve='convex', direction='decreasing')
rf_selected = importances.iloc[:kl.knee].index
```

### 📋 **Template 5: LASSO Selection**
```python
from sklearn.linear_model import LogisticRegression

lasso = LogisticRegression(penalty='l1', solver='liblinear', C=0.1, random_state=42)
lasso.fit(X_train, y_train['RFS_STATUS'])
lasso_selected = X_train.columns[lasso.coef_[0] != 0]
```

### 📋 **Template 6: Boruta**
```python
from boruta import BorutaPy

rf = RandomForestClassifier(n_jobs=-1, max_depth=5, random_state=42)
boruta = BorutaPy(rf, n_estimators=100, max_iter=100, random_state=42)
boruta.fit(X_train.values, y_train['RFS_STATUS'].values)
boruta_selected = X_train.columns[boruta.support_]
```

### 📋 **Template 7: Cox Regression**
```python
from lifelines import CoxPHFitter

cox_results = {}
for gene in X_train.columns:
    df = pd.DataFrame({
        'duration': y_train['RFS_MONTHS'],
        'event': y_train['RFS_STATUS'],
        'gene': X_train[gene]
    })
    cph = CoxPHFitter()
    cph.fit(df, duration_col='duration', event_col='event')
    cox_results[gene] = {
        'hr': cph.hazard_ratios_['gene'],
        'p': cph.summary.loc['gene', 'p']
    }
```

### 📋 **Template 8: Ensemble Voting**
```python
# Create voting matrix
all_features = set().union(*[set(fs) for fs in all_feature_sets])
vote_matrix = pd.DataFrame(0, index=all_features, columns=method_names)
for method, features in zip(method_names, all_feature_sets):
    vote_matrix.loc[features, method] = 1

vote_counts = vote_matrix.sum(axis=1)
majority_features = vote_counts[vote_counts >= 4].index  # Selected by ≥4 methods
```

## 🎯 Learning Assessment

### ✅ **Self-Check Questions**

After completing the feature selection activities, you should be able to answer:

1. **Advanced Variance Filtering**
   - How does this differ from the variance filtering in preprocessing (notebook 2)?
   - When should you use stricter variance thresholds?
   - What are the trade-offs between aggressive vs conservative variance filtering?

2. **De-correlation**
   - Why is removing highly correlated features important for some ML algorithms?
   - How do you decide which feature to keep when two are highly correlated?
   - What correlation threshold is appropriate for your data?

3. **Differential Expression**
   - What is the difference between fold change and statistical significance?
   - Why is multiple testing correction necessary?
   - How do you interpret log2 fold change values?

4. **Random Forest with KneeLocator**
   - What does the "knee" point represent in feature importance curves?
   - How does RF importance differ from permutation importance?
   - What are advantages of automatic threshold selection via KneeLocator?

5. **LASSO and SVM Selection**
   - How does L1 regularization lead to feature selection?
   - What is the relationship between C parameter and number of selected features?
   - When should you use LASSO vs SVM for feature selection?

6. **Boruta Algorithm**
   - What makes Boruta "all-relevant" vs "minimal-optimal" feature selection?
   - How do shadow features help identify important features?
   - How should you handle tentative features?

7. **Cox Regression**
   - What does a hazard ratio > 1 indicate for gene expression?
   - How is survival-based selection different from classification-based selection?
   - Why are both RFS_MONTHS and RFS_STATUS needed?

8. **Method Comparison**
   - Which methods tend to select similar features?
   - What are advantages of ensemble feature selection?
   - How do you balance feature set size vs predictive performance?

### 🏆 **Success Criteria**

You have successfully completed this notebook if you can:
- ✅ Apply all 7 feature selection methods independently
- ✅ Understand the theoretical basis of each method
- ✅ Interpret results from statistical and ML-based approaches
- ✅ Compare and contrast different selection strategies
- ✅ Create ensemble feature sets using voting
- ✅ Validate feature selections on held-out data
- ✅ Export optimized feature sets for downstream modeling
- ✅ Make informed decisions about which features to use

### 🚀 **Extension Challenges** (Optional)

For advanced students:
1. Implement stability selection using bootstrap resampling
2. Use SHAP values for feature importance ranking
3. Apply recursive feature elimination with cross-validation (RFECV)
4. Integrate pathway/network information into feature selection
5. Implement multi-objective optimization (accuracy vs interpretability vs cost)

---

## 📚 Feature Selection Summary

In this notebook, you have successfully completed:

### ✅ **Completed Tasks:**
1. **Data Loading**: Loaded preprocessed data from notebook 2
2. **Advanced Variance Filtering**: Applied stricter thresholds than preprocessing
3. **De-correlation**: Removed redundant highly correlated features
4. **Differential Expression**: Identified category-specific genes using statistical tests
5. **Random Forest + KneeLocator**: Used elbow method for optimal feature selection
6. **ML-Based Selection**: Applied LASSO and SVM for embedded feature selection
7. **Boruta Algorithm**: Identified all-relevant features using shadow features
8. **Cox Regression**: Found survival-associated genes
9. **Method Comparison**: Created ensemble feature sets from all 7 methods

### 🎯 **Key Achievements:**
- **Comprehensive Analysis**: Applied 7 complementary feature selection methods
- **Statistical Rigor**: Used multiple testing correction and proper thresholds
- **Automated Optimization**: Used KneeLocator for data-driven threshold selection
- **Survival Integration**: Connected gene expression to patient outcomes
- **Ensemble Strategy**: Combined methods for robust feature sets
- **Validation**: Tested feature sets on held-out validation data

### 🔄 **Comparison with Previous Notebooks:**
- **Notebook 1 (Exploration)**: Understood data structure and quality
- **Notebook 2 (Preprocessing)**: Basic filtering (variance ~0.01), normalization, train/test split
- **Notebook 3 (Feature Selection)**: **Advanced filtering** (variance >0.1), statistical testing, ML-based ranking, ensemble selection

### 📁 **Exported Files:**
- `../results/feature_selection/`: All method results and rankings
- `../results/feature_selection/ensemble_minimal.csv`: High-confidence features
- `../results/feature_selection/ensemble_balanced.csv`: Recommended for modeling
- `../results/feature_selection/ensemble_comprehensive.csv`: Maximum coverage
- `../results/feature_selection/method_comparison.json`: Complete analysis report

### 🔄 **Next Steps:**
In the next notebook (`04_model_development.ipynb`), we will:
1. **Train ML models** using the selected feature sets
2. **Compare algorithms**: Random Forest, SVM, Neural Networks, etc.
3. **Hyperparameter tuning**: Optimize model performance
4. **Clinical evaluation**: Assess models using survival metrics
5. **Model interpretation**: Understand feature contributions to predictions

---

**Excellent work completing advanced feature selection! 🎉**

You've successfully applied 7 state-of-the-art methods and created optimized feature sets that balance statistical rigor, predictive power, and biological interpretability. These refined feature sets will serve as the foundation for robust machine learning models in the next notebook.